In [32]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [33]:
from transformers import BlipProcessor, BlipForConditionalGeneration, BertModel, BertTokenizer
from PIL import Image
import requests
import torch

In [34]:
# Инициализация модели и процессора
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

d:\Users\asas7\miniconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
# Инициализация BERT модели и токенизатора
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [36]:
# Загрузка изображения
image = Image.open(requests.get("https://fresco.wallset.ru/images/detailed/1208/3086.jpg", stream=True).raw)

In [37]:
# Подготовка изображения для модели
inputs = processor(image, return_tensors="pt")

In [38]:
# Вывод тензорного представления изображения
print("Тензорное представление изображения:")
print(inputs)

Тензорное представление изображения:
{'pixel_values': tensor([[[[-1.2375, -1.2667, -1.2667,  ..., -1.1645, -1.2083, -1.2083],
          [-1.2521, -1.2667, -1.2521,  ..., -1.1353, -1.1791, -1.1937],
          [-1.2375, -1.2229, -1.2521,  ..., -1.0331, -1.1645, -1.1937],
          ...,
          [-1.3543, -1.3543, -1.3835,  ..., -0.9164, -0.8142, -0.9602],
          [-1.3689, -1.3689, -1.3835,  ..., -1.0623, -0.8142, -0.8434],
          [-1.3543, -1.3543, -1.3543,  ..., -1.1645, -1.0331, -0.7704]],

         [[-1.1518, -1.1968, -1.1968,  ..., -1.0767, -1.1218, -1.1368],
          [-1.1668, -1.1818, -1.1818,  ..., -1.0317, -1.0918, -1.1218],
          [-1.0918, -1.1068, -1.1668,  ..., -0.8816, -1.0467, -1.1068],
          ...,
          [-1.3319, -1.3319, -1.3619,  ..., -0.6115, -0.4764, -0.3564],
          [-1.3469, -1.3469, -1.3619,  ..., -0.7616, -0.4314, -0.1913],
          [-1.3319, -1.3319, -1.3319,  ..., -0.9117, -0.6415, -0.1313]],

         [[-0.9541, -1.0110, -1.0252,  ..., -0.9

In [39]:
# Генерация описания изображения
out = model.generate(**inputs)

d:\Users\asas7\miniconda3\Lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [40]:
# Вывод тензорного представления сгенерированного описания
print("Тензорное представление сгенерированного описания:")
print(out)
print(out.shape)

Тензорное представление сгенерированного описания:
tensor([[30522,  2048,  2317,  7600, 10201,  1999,  1996,  5568,   102]])
torch.Size([1, 9])


In [41]:
# Декодирование описания
description = processor.decode(out[0], skip_special_tokens=True)
print("Описание изображения:")
print(description)

Описание изображения:
two white tigers laying in the grass


In [42]:
# Получение векторного представления описания с помощью BERT
with torch.no_grad():
    encoded_input = bert_tokenizer(description, return_tensors="pt", padding=True, truncation=True)
    outputs = bert_model(**encoded_input)
    hidden_states = outputs.last_hidden_state

In [43]:
# Вывод векторного представления описания
print("Векторное представление описания:")
print(hidden_states)
print(hidden_states.shape)

Векторное представление описания:
tensor([[[-0.5840,  0.0341, -1.0941,  ..., -0.1140,  0.3301, -0.4560],
         [-0.1048, -0.2508, -0.8683,  ..., -0.0879,  0.0358, -0.3640],
         [ 0.1130,  0.1523, -0.6842,  ..., -0.4460,  0.3589, -0.5190],
         ...,
         [ 0.1637, -0.5750, -0.6242,  ..., -0.5067,  0.2107, -0.3126],
         [-0.5201, -0.0070, -0.7997,  ...,  0.3047,  0.3784, -0.3294],
         [ 0.8096,  0.0042, -0.4417,  ...,  0.1063, -0.4618, -0.4010]]])
torch.Size([1, 9, 768])


In [45]:
import torch

In [50]:
abs_tensor = torch.abs(hidden_states)
summed_tensor = abs_tensor.sum(dim=2)
print(summed_tensor)
print(summed_tensor.shape)

tensor([[295.5405, 272.2629, 270.0369, 269.5868, 266.3513, 313.6562, 307.0774,
         316.4016, 286.2222]])
torch.Size([1, 9])
